Importing the model from Data folder

In [1]:
from cobra.io import read_sbml_model

In [2]:
model = read_sbml_model('data/iYO844.xml')

Show the model stats

In [3]:
model

Name,iYO844
Memory address,0x026ec17dd130
Number of metabolites,990
Number of reactions,1250
Number of groups,0
Objective expression,1.0*BIOMASS_BS_10 - 1.0*BIOMASS_BS_10_reverse_8788b
Compartments,"cytosol, extracellular space"


Adding surfactin production to the model

In [4]:
from cobra import Reaction, Metabolite

Will add 4 different reactions due to there being 4 different fatty acids that can make up the fatty acid part of surfactin 

In [5]:
new_reaction = Reaction('SP_1') #Surfactin Production reaction 1 

In [6]:
surfactin = Metabolite(id='surfactin_c', compartment='c') 

In [7]:
new_reaction.add_metabolites({model.metabolites.glu__L_c: -1,
                              model.metabolites.leu__L_c: -4,
                              model.metabolites.val__L_c:-1, 
                              model.metabolites.asp__L_c: -1,
                              model.metabolites.nadph_c:-1,
                              model.metabolites.get_by_id('3hddcoa_c'):-1,
                              model.metabolites.atp_c:-1,
                              model.metabolites.adp_c:1,
                              model.metabolites.nadp_c:1,
                              surfactin: 1 
                             })

In [8]:
model.add_reactions([new_reaction])

In [ ]:
new_reaction2 = Reaction('SP_2') 

In [13]:
new_reaction2.add_metabolites({model.metabolites.glu__L_c: -1,
                              model.metabolites.leu__L_c: -4,
                              model.metabolites.val__L_c:-1, 
                              model.metabolites.asp__L_c: -1,
                              model.metabolites.nadph_c:-1,
                              model.metabolites.tdcoa_c:-1,
                              model.metabolites.atp_c:-1,
                              model.metabolites.adp_c:1,
                              model.metabolites.nadp_c:1,
                              surfactin: 1 
                             })

In [14]:
model.add_reactions([new_reaction2])

In [15]:
new_reaction3= Reaction('SP_3')

In [16]:
new_reaction3.add_metabolites({model.metabolites.glu__L_c: -1,
                              model.metabolites.leu__L_c: -4,
                              model.metabolites.val__L_c:-1, 
                              model.metabolites.asp__L_c: -1,
                              model.metabolites.nadph_c:-1,
                              model.metabolites.get_by_id('3hhdcoa_c'):-1,
                              model.metabolites.atp_c:-1,
                              model.metabolites.adp_c:1,
                              model.metabolites.nadp_c:1,
                              surfactin: 1 
                             })

In [17]:
model.add_reactions([new_reaction3])

In [18]:
new_reaction4= Reaction('SP_4')

In [19]:
new_reaction4.add_metabolites({model.metabolites.glu__L_c: -1,
                              model.metabolites.leu__L_c: -4,
                              model.metabolites.val__L_c:-1, 
                              model.metabolites.asp__L_c: -1,
                              model.metabolites.nadph_c:-1,
                              model.metabolites.get_by_id('3ohdcoa_c'):-1,
                              model.metabolites.atp_c:-1,
                              model.metabolites.adp_c:1,
                              model.metabolites.nadp_c:1,
                              surfactin: 1 
                             })

In [20]:
model.add_reactions([new_reaction4])

Time to look at production of surfactin

In [10]:
model.add_boundary(model.metabolites.surfactin_c, type='demand') #forcing surfactin production

Reaction identifier,DM_surfactin_c
Name,demand
Memory address,0x026ec76c8c70
Stoichiometry,surfactin_c --> -->
GPR,
Lower bound,0
Upper bound,1000.0


In [11]:
#flux for reaction 1
with model:
    model.objective = model.reactions.SP_1
    print(model.optimize().objective_value)

0.09298767530811763


The flux for SP_1 is calculated to 0.09

In [21]:
# Remove?
with model:
    model.objective = model.reactions.SP_2
    print(model.optimize().objective_value)

0.08967213114754109


In [22]:
# Remove?
with model:
    model.objective = model.reactions.SP_3
    print(model.optimize().objective_value)

0.0873918253228599


In [23]:
# Remove?
with model:
    model.objective = model.reactions.SP_4
    print(model.optimize().objective_value)

0.08780096308186207


In [24]:
#Flux for surfactin production, so all reactions included 
with model:
    model.objective = model.reactions.DM_surfactin_c
    print(model.optimize().objective_value)

0.0929876753081174


Changing the carbon source to sucrose

In [25]:
model.metabolites.query('suc') #finding sucrose in model - remove?

[<Metabolite argsuc_c at 0x26ec57a9f10>,
 <Metabolite suc6p_c at 0x26ec5971b80>,
 <Metabolite sucbz_c at 0x26ec5971df0>,
 <Metabolite succ_c at 0x26ec5971fa0>,
 <Metabolite succ_e at 0x26ec5971280>,
 <Metabolite succoa_c at 0x26ec5971490>,
 <Metabolite suchms_c at 0x26ec59713a0>,
 <Metabolite sucr_c at 0x26ec59883d0>,
 <Metabolite sucr_e at 0x26ec59884f0>,
 <Metabolite sucsal_c at 0x26ec59886d0>]

In [26]:
model.metabolites.sucr_e

Metabolite identifier,sucr_e
Name,Sucrose C12H22O11
Memory address,0x026ec59884f0
Formula,C12H22O11
Compartment,e
In 2 reaction(s),"SUCpts, EX_sucr_e"


In [27]:
model.reactions.EX_glc__D_e.lower_bound = 0

In [28]:
model.reactions.EX_sucr_e.lower_bound = -1000

In [29]:
model.optimize()

,fluxes,reduced_costs
EX_etha_e,0.0,-2.496739e-01
EX_drib_e,0.0,-1.548569e-16
EX_csn_e,0.0,-7.490218e-01
EX_5mtr_e,0.0,0.000000e+00
EX_dtmp_e,0.0,-4.993479e-01
...,...,...
SP_1,0.0,-1.747717e+00
DM_surfactin_c,0.0,0.000000e+00
SP_2,0.0,-1.747717e+00
SP_3,0.0,-1.747717e+00


In [30]:
model.reactions.EX_sucr_e.flux

-3.832839306087132

Calculating the yield

In [33]:
model.objective = model.reactions.DM_surfactin_c
surfactin_production = model.optimize().objective_value
print('Maximum productivity =', surfactin_production, 'mmol/gDW*h')
   
maximum_yield = surfactin_production / (-1*(model.reactions.EX_sucr_e.flux))
print('Maximum theoretical yield =', maximum_yield, 'mmol-surfactin/mmol-sucrose')

Maximum productivity = 0.35714285714285604 mmol/gDW*h
Maximum theoretical yield = 0.12549488309554035 mmol-surfactin/mmol-sucrose


The article had 0.154 mol-surfactin/mol-sucrose after knock-in. 

Sandie starts doing stuff from here:

Compute gene knockout strategies using algorithms like OptKnock, OptGene and OptCouple